# Profiling
Sometimes computing the likelihood is not as fast as we would like. Theano provides handy profiling tools which are wrapped in PyMC3 by `model.profile`. This function returns a `ProfileStats` object conveying information about the underlying Theano operations. Here we'll profile the likelihood and gradient for the stochastic volatility example.

First we build the model.

In [ ]:
import numpy as np
import pandas as pd
import pymc3 as pm

print(f"Running on PyMC3 v{pm.__version__}")

In [ ]:
# Load the data
try:
    returns = pd.read_csv("../data/SP500.csv", index_col=0, parse_dates=True)
except FileNotFoundError:
    returns = pd.read_csv(pm.get_data("SP500.csv"), index_col=0, parse_dates=True)

In [ ]:
# Stochastic volatility example
with pm.Model() as model:
    sigma = pm.Exponential("sigma", 1.0 / 0.02, testval=0.1)
    nu = pm.Exponential("nu", 1.0 / 10)
    s = pm.GaussianRandomWalk("s", sigma ** -2, shape=returns.shape[0])
    r = pm.StudentT("r", nu, lam=np.exp(-2 * s), observed=returns["change"])

Then we call the `profile` function and summarize its return values.

In [ ]:
# Profiling of the logp call
model.profile(model.logpt).summary()

In [ ]:
# Profiling of the gradient call dlogp/dx
model.profile(pm.gradient(model.logpt, model.vars)).summary()

In [ ]:
%load_ext watermark
%watermark -n -u -v -iv -w -p theano,xarray